In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb 
pd.set_option('display.max_columns', 200)

In [2]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

C:\Users\DELL\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3427: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
def split_loan(x):
    x = str(x)
    x = x.replace('and', '')
    y = x.split(',')
    z = [i.replace(' ', '') for i in y]
    return z

data['Type_of_Loan'] = data['Type_of_Loan'].apply(lambda x: split_loan(x))
test['Type_of_Loan'] = test['Type_of_Loan'].apply(lambda x: split_loan(x))

loans = ['AutoLoan',
 'Credit-BuilderLoan',
 'DebtConsolidationLoan',
 'HomeEquityLoan',
 'MortgageLoan',
 'NotSpecified',
 'PaydayLoan',
 'PersonalLoan',
 'StudentLoan',
 'nan']

for col in loans:
    
    data[col] = data['Type_of_Loan'].apply(lambda x: x.count(col))
    test[col] = test['Type_of_Loan'].apply(lambda x: x.count(col))

In [4]:
Occupation = ['Scientist', '_______', 'Teacher', 'Engineer', 'Entrepreneur',
       'Developer', 'Lawyer', 'Media_Manager', 'Doctor', 'Journalist',
       'Manager', 'Accountant', 'Musician', 'Mechanic', 'Writer',
       'Architect']

Occupation_dict = {}
for i, o in enumerate(Occupation):
    Occupation_dict[o] = i

data['Occupation'] = data['Occupation'].map(Occupation_dict)
test['Occupation'] = test['Occupation'].map(Occupation_dict)

In [5]:
data['Age'] = data['Age'].apply(lambda x : x.replace('_', ''))
test['Age'] = test['Age'].apply(lambda x : x.replace('_', ''))

data['Age'] = data['Age'].astype(float)
test['Age'] = test['Age'].astype(float)

In [6]:
data['Annual_Income'] = data['Annual_Income'].apply(lambda x : float(x[:-1]) if x[-1] == '_' else float(x))
test['Annual_Income'] = test['Annual_Income'].apply(lambda x : float(x[:-1]) if x[-1] == '_' else float(x))

In [7]:
data['Interest_Rate'] = data['Interest_Rate'].apply(lambda x : int(str(x)[0]))
test['Interest_Rate'] = test['Interest_Rate'].apply(lambda x : int(str(x)[0]))

In [8]:
data['Num_of_Loan'] = data['Num_of_Loan'].apply(lambda x : -1 if x =='-100' else int(x[0]))
test['Num_of_Loan'] = test['Num_of_Loan'].apply(lambda x : -1 if x =='-100' else int(x[0]))

In [9]:
data['Num_of_Delayed_Payment'] = data['Num_of_Delayed_Payment'].fillna('-999')
test['Num_of_Delayed_Payment'] = test['Num_of_Delayed_Payment'].fillna('-999')

data['Num_of_Delayed_Payment'] = data['Num_of_Delayed_Payment'].apply(lambda x : float(x[:-1]) if x[-1] == '_' else float(x))
test['Num_of_Delayed_Payment'] = test['Num_of_Delayed_Payment'].apply(lambda x : float(x[:-1]) if x[-1] == '_' else float(x))

In [10]:
data['Changed_Credit_Limit'] = data['Changed_Credit_Limit'].apply(lambda x : x.replace('_', '-999.99'))
test['Changed_Credit_Limit'] = test['Changed_Credit_Limit'].apply(lambda x : x.replace('_', '-999.99'))

data['Changed_Credit_Limit'] = data['Changed_Credit_Limit'].apply(lambda x : float(x[:-1]) if x[-1] == '_' else float(x))
test['Changed_Credit_Limit'] = test['Changed_Credit_Limit'].apply(lambda x : float(x[:-1]) if x[-1] == '_' else float(x))

In [11]:
data['Num_Credit_Inquiries'] = data['Num_Credit_Inquiries'].fillna('-999.99')
test['Num_Credit_Inquiries'] = test['Num_Credit_Inquiries'].fillna('-999.99')

data['Num_Credit_Inquiries'] = data['Num_Credit_Inquiries'].apply(lambda x: float(str(x)[0] if float(x) >= 0.0 else x))
test['Num_Credit_Inquiries'] = test['Num_Credit_Inquiries'].apply(lambda x: float(str(x)[0] if float(x) >= 0.0 else x))

In [12]:
Credit_Mix = {'Bad' : -1, 'Good' : 1, 'Standard' : 0, '_' : 0}

data['Credit_Mix'] = data['Credit_Mix'].map(Credit_Mix)
test['Credit_Mix'] = test['Credit_Mix'].map(Credit_Mix)

In [13]:
data['Outstanding_Debt'] = data['Outstanding_Debt'].apply(lambda x : float(x[:-1]) if x[-1] == '_' else float(x))
test['Outstanding_Debt'] = test['Outstanding_Debt'].apply(lambda x : float(x[:-1]) if x[-1] == '_' else float(x))

In [14]:
data['Credit_History_Age'] = data['Credit_History_Age'].fillna('0 Years and 0 Months').apply(lambda x :  100 * int(x.split('Years')[0]) + int(x.split('and')[1].replace('Months','')))
test['Credit_History_Age'] = test['Credit_History_Age'].fillna('0 Years and 0 Months').apply(lambda x :  100 * int(x.split('Years')[0]) + int(x.split('and')[1].replace('Months','')))

In [15]:
Payment_of_Min_Amount = {'No' : -1, 'Yes' : 1, 'NM' : 0}

data['Payment_of_Min_Amount'] = data['Payment_of_Min_Amount'].map(Payment_of_Min_Amount)
test['Payment_of_Min_Amount'] = test['Payment_of_Min_Amount'].map(Payment_of_Min_Amount)

In [16]:
data['Amount_invested_monthly'] = data['Amount_invested_monthly'].fillna('-999.99')
test['Amount_invested_monthly'] = test['Amount_invested_monthly'].fillna('-999.99')

data['Amount_invested_monthly'] = data['Amount_invested_monthly'].apply(lambda x : x.replace('_', ''))
test['Amount_invested_monthly'] = test['Amount_invested_monthly'].apply(lambda x : x.replace('_', ''))

data['Amount_invested_monthly'] = data['Amount_invested_monthly'].astype(float)
test['Amount_invested_monthly'] = test['Amount_invested_monthly'].astype(float)

In [17]:
Payment_Behaviour = {'!@9#%8' : 0,
 'High_spent_Large_value_payments' : 1,
 'High_spent_Medium_value_payments' : 2,
 'High_spent_Small_value_payments' : 3,
 'Low_spent_Large_value_payments' : 4,
 'Low_spent_Medium_value_payments' : 5,
 'Low_spent_Small_value_payments' : 6}

data['Payment_Behaviour'] = data['Payment_Behaviour'].map(Payment_Behaviour)
test['Payment_Behaviour'] = test['Payment_Behaviour'].map(Payment_Behaviour)

In [18]:
data['Monthly_Balance'] = data['Monthly_Balance'].fillna('-999.99')
test['Monthly_Balance'] = test['Monthly_Balance'].fillna('-999.99')

data['Monthly_Balance'] = data['Monthly_Balance'].apply(lambda x : str(x).replace('_', ''))
test['Monthly_Balance'] = test['Monthly_Balance'].apply(lambda x : str(x).replace('_', ''))

data['Monthly_Balance'] = data['Monthly_Balance'].astype(float)
test['Monthly_Balance'] = test['Monthly_Balance'].astype(float)

In [19]:
data['Credit_Score'] = data['Credit_Score'].map({'Poor': 0, 'Standard': 1, 'Good': 2})

In [20]:
del_cols = ['ID', 'Customer_ID', 'Month','Name', 'SSN', 'Type_of_Loan',]
cat_cols = ['Occupation', 'Payment_Behaviour']
num_cols = ['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 
           'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
           'Num_Credit_Inquiries', 'Credit_Mix','Outstanding_Debt','Credit_Utilization_Ratio', 'Credit_History_Age',
           'Payment_of_Min_Amount','Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance']

# Monthly_Inhand_Salary fillna requires

cols = cat_cols + num_cols + loans

In [21]:
data['split'] = data['Month'].apply(lambda x: 1 if x in ['July', 'August'] else 0)

In [22]:
train = data[data['split'] == 0]
valid = data[data['split'] == 1]

train_dataset = lgb.Dataset(train[cols], train['Credit_Score'])
valid_dataset = lgb.Dataset(valid[cols], valid['Credit_Score'])

In [25]:
lgb_params = {
    'boosting_type': 'gbdt',
    #'objective': 'multiclass',
    'objective': 'ovr',
    #'metric': 'ovr',
    'num_class' : 3,
    #'num_iterations' : 4000,
    #'learning_rate' : 0.05,
    #'min_data_in_leaf' : 75,
    #'num_leaves' : 71,
    'categorical_feature' : [0,1],
     #'feature_fraction' : 0.5,
    #'bagging_fraction' : 0.8,
    #'bagging_freq' : 1,
    #'lambda_l1' : 0.0,
    #'lambda_l2' : 0.0,
    'nthread': 1,
    'seed' : 16,
}

In [26]:
model = lgb.train(params = lgb_params,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, valid_dataset], 
                          verbose_eval = 50,
                          early_stopping_rounds=50)

[LightGBM] [Info] Number of positive: 21782, number of negative: 53218
[LightGBM] [Info] Number of positive: 40214, number of negative: 34786
[LightGBM] [Info] Number of positive: 13004, number of negative: 61996
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025006 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3493
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.290427 -> initscore=-0.893313
[LightGBM] [Info] Start training from score -0.893313
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.536187 -> initscore=0.145000
[LightGBM] [Info] Start training from score 0.145000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.173387 -> initscore=-1.561813
[LightGBM] [Info] Start training from score -1.561813
Training until validation scores don't improve for 50 rounds
[50]	training's multi_logloss: 0.5

In [29]:
preds = model.predict(test[cols])

In [33]:
test['Credit_Score'] = np.argmax(preds, axis = 1)
test['Credit_Score'] = test['Credit_Score'].map({0:'Poor', 1:'Standard', 2:'Good'})

In [35]:
test[['ID', 'Credit_Score']].to_csv('sub0.csv', index = False)